In [1]:
#OLLAMAPool Server Prototype Code
import os
EndPoint_Queries=os.environ.get('EndPoint_Queries')
Endpoint_Results=os.environ.get('Endpoint_Results')
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_Queries is None:
    raise ValueError("EndPoint_Queries is not set")
if Endpoint_Results is None:
    raise ValueError("Endpoint_Results is not set")
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
print("Environment Variables are set OK")

Environment Variables are set OK


In [25]:
#Busines Classes

import json
from typing import List
import uuid
from azure.servicebus import ServiceBusClient, ServiceBusMessage

class LLMRequest:
    def __init__(self,Model:str="",systemMessage:str="",query:str=""):
        self.UUID=str(uuid.uuid4())
        self.Model=Model
        self.systemMessage=systemMessage
        self.query=query

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
        
class LLMResult:
    def __init__(self,UUID:str="",result:str="",errorMsg:str="",timeDelta:str=""):
        self.UUID=UUID
        self.result=result
        self.errorMsg=errorMsg
        self.HasError=errorMsg!=""
        self.timeDelta=timeDelta

    def to_json(self):
        return self.__dict__
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)
    

In [26]:
#Ollama Processing Code
import datetime
from ollama import Client
client = Client(host='http://localhost:11434')
#client.pull("llama3.1")

def ProcessLLMRequest(request:LLMRequest)->LLMResult:
    print("Processing Request "+request.UUID)
    print(request.to_json())
    #ret=client.generate(request.query)
    
    #Start timer
    timerStart=datetime.datetime.now()
    
    
    ret = client.chat(
        model='llama3.1',
        messages=[{'role': 'system', 'content': request.systemMessage},
        {'role': 'user', 'content': request.query}],
        stream=False)
    
    timerEnd=datetime.datetime.now()
    timeDelta=timerEnd-timerStart
    
    result=LLMResult(UUID=request.UUID,result=ret,timeDelta=str(timeDelta))
    return result
    


In [30]:
#Main Message Handling Loop
import time
from typing import List
from azure.servicebus import ServiceBusClient, ServiceBusMessage

# Replace with your connection string and queue name
queue_name = "llm-queries"

# Create a Service Bus client
servicebus_client = ServiceBusClient.from_connection_string(conn_str=EndPoint_Queries)

def receive_messages_from_queue():
    # Create a receiver for the queue
    with servicebus_client.get_queue_receiver(queue_name=queue_name) as receiver:
        print("Receiving messages from the queue...")
        
        # Receive messages in a batch, you can specify max_message_count as needed
        received_msgs = receiver.receive_messages(max_message_count=1, max_wait_time=5)

        for msg in received_msgs:
            # Print the message payload
            print(f"Received message: {str(msg)}")
            
            llmRequest=LLMRequest()
            llmRequest.from_json(str(msg))
            print(llmRequest.query)
            result=ProcessLLMRequest(llmRequest)
            print(result.to_json())
            
            # Accept the message to remove it from the queue
            receiver.complete_message(msg)

# Call the function to read from the queue
receive_messages_from_queue()


Receiving messages from the queue...
Received message: {"UUID": "5980331b-2dc3-49b2-9b6a-fee4e25eba8c", "Model": "llama3.1", "systemMessage": "You are a science fiction fantasy writer writing beautifully creative descriptions from a list of character traits. \nYou have a list of character traits, be creative and expand on this (going so far as to write a short background story if you wish).\n", "query": "They have the followng traits:\nrace: halfling\ncomplection: pale\nheight: tall\nbuild: muscular\nfeature: sharp\neye: dull\nhair: wavy\nclothes: tattered\nsophistication: refined\nmannerism: boisterous\nvirtue: kind\n"}
They have the followng traits:
race: halfling
complection: pale
height: tall
build: muscular
feature: sharp
eye: dull
hair: wavy
clothes: tattered
sophistication: refined
mannerism: boisterous
virtue: kind

Processing Request 5980331b-2dc3-49b2-9b6a-fee4e25eba8c
{'UUID': '5980331b-2dc3-49b2-9b6a-fee4e25eba8c', 'Model': 'llama3.1', 'systemMessage': 'You are a science fi